# Notes on Coiltraine Baseline Experiment

## Experiment Setup

Some changes have to be performed given the outdated versions of the packages used in the original experiment. These extra details about the setup are described below:

### Code, datasets and libraries

- Clone repository

```bash
$ git clone https://github.com/felipecode/coiltraine.git
```

- Set dataset path env variable

```bash
$ export COIL_DATASET_PATH=/home/farnez/Projects/data/CARLA100
```

- Download datasets with scripts from "Tools" Folder

```bash
$ python3 get_baseline_dataset.py
$ python3 get_offline_online_data.py
$ python3 get_samples_datasets.py
$ python3 get_town03_dataset.py
```

- Install python packages requirements:

    ```bash
    $ pip install <package name>
    ```
    - List of packages:
        
        - numpy==1.19.0
        - torch==1.5.0
        - torchvision==0.6.0
        - tensorflow==2.2.9
        - tensorboard==2.2.2
        - tensorflow-tensorboard==1.5.1 [Link tensorboard package issue](https://github.com/tensorflow/tensorboard/issues/1862)
        - scipy==1.0.1
        - pillow==5.3.0
        - scikit-image==0.14.2
        - imgaug==0.3.0
        

### File Changes
Edit some files to match the current version of installed packages:

- Edit "__tensorbaord_logger.py__" file (../logger/tensorbaord_logger.py)

    Replace __*tf.summary.FileWriter()*__ Method with __*tf.summary.create_file_writer()*__ method [[Link 1](https://github.com/eriklindernoren/PyTorch-YOLOv3/issues/327)][[Link 2](https://github.com/eriklindernoren/PyTorch-YOLOv3/commit/930c5872eea3d22233805f3f693dd5c9a9892707)]

    ```python
    class Logger(object):

    def __init__(self, log_dir):
        """Create a summary writer logging to log_dir."""
        #from datetime import datetime
        #now = datetime.now()
        #log_dir = log_dir + now.strftime("%Y%m%d-%H%M%S")
        # self.writer = tf.summary.FileWriter(log_dir)
        self.writer = tf.summary.create_file_writer(log_dir)

    ```

- __Edit the docker subprocess call...__

    current implementation uses a deprecated command (--runtime)

    - Edit "__run_drive.py__" file (../coil_core/run_drive.py):

    ```python
    class Logger(object):
    sp = subprocess.Popen(['docker', 'run', '--rm', '-d', '-p',
                       str(port)+'-'+str(port+2)+':'+str(port)+'-'+str(port+2),
                       '--runtime=nvidia', '-e', 'NVIDIA_VISIBLE_DEVICES='+str(gpu), docker,
                       '/bin/bash', 'CarlaUE4.sh', '/Game/Maps/' + town_name, '-windowed',
                       '-benchmark', '-fps=10', '-world-port=' + str(port)], shell=False,
                       stdout=subprocess.PIPE)
    ```
    - Current command for CarlaSimv0.8.4 docker container:

    ```bash
    $ docker run -p 2000-2002:2000-2002 --gpus all carla_sim_ubuntu16:0.8.4 /bin/bash ./CarlaUE4.sh
    ```

    - Docker container command from CARLA docs 0.8.4:
    ```bash
    $ docker run -p 2000-2002:2000-2002 --runtime=nvidia -e NVIDIA_VISIBLE_DEVICES=0 carlasim/carla:0.8.4 /bin/bash CarlaUE4.sh  < Your list of parameters >
    ```
    The parameter "--runtime" is deprecated!

    - __Final modification to this line of code__:

    ```python
        sp = subprocess.Popen(['docker', 'run', '--rm', '-d', '-p',
                               str(port)+'-'+str(port+2)+':'+str(port)+'-'+str(port+2),
                               '--gpus', 'all', docker,
                               '/bin/bash', 'CarlaUE4.sh', '/Game/Maps/' + town_name, '-windowed',
                               '-benchmark', '-fps=10', '-world-port=' + str(port)], shell=False,
                               stdout=subprocess.PIPE)
    ```

- Edit "__city_track.py__" fiel to match later version of numpy (../carla08/planner/city_track.py) [[Link - issues with numpy](https://github.com/numpy/numpy/issues/12987)]

    Change the "threshold" parameter value

```python
        import sys
        ...
        ...
        ...
        # np.set_printoptions(edgeitems=3, infstr='inf', threshold=np.nan, linewidth=129)
        np.set_printoptions(edgeitems=3, infstr='inf', threshold=sys.maxsize, linewidth=129)
```

__Test the best trained model in the hardes environment__:

```bash
$ python3 coiltraine.py --gpus 0 --single-process drive -e resnet34imnet10S2 --folder nocrash \
-de NocrashNewWeatherTown_Town02 --docker carla_sim_ubuntu16:0.8.4
```

- __carla_sim_ubuntu16:0.8.4__  is the docker-image that will be executed for testing

## Documentation Guide

### Configuration

1. __Configuration__ [[Link](https://github.com/felipecode/coiltraine/blob/master/docs/configuration.md)]

    The COiLTRAiNE framework works with a central global module with all the default configurations. The configurations set data related things (dataset, input format), network architecture, training hyperparameters and others.

    When executing any process, it first merges a configuration file with the central global module.

    Files/Batches
    The configuration file is defined as a YAML files. Each file is associated with a exp batch, a folder grouping a set of exp alias that is the name of the YAML file. All exp batches are located inside the configs folder.
    
    __Files and Batches__
    
    The configuration file is defined as a YAML files. Each file is associated with a exp batch, a folder grouping a set of exp alias that is the name of the YAML file. All exp batches are located inside the configs folder.

    A fully commented configuration file can be found at:
    
    ```bash
    $ ~/../coiltraine/configs/sample/resnet34.yaml
    ```

### Core Modules

2. __Core Modules__ [[Link](https://github.com/felipecode/coiltraine/blob/master/docs/main_modules.md)]
    
    There are three kinds of process that are executed on COiLTRAiNE: Train, Validate, Drive

    1. __TRAIN__
    
    Trains a controller model given a configuration file. The training process produces checkpoints with respect to the SAVE_SCHEDULE attribute. These checkpoints are used to evaluate the model on a dataset or in some driving benchmark.

    The training produces logs to be printed on the screen and also tensorboard logs.
    
    __Train configuration__:
    
        - Model configurations [[Link](https://github.com/felipecode/coiltraine/blob/master/docs/network.md)] 
            
        - Input Data [[Link](https://github.com/felipecode/coiltraine/blob/master/docs/input.md)]
    
        Run the following command to train the model "__~/.../coiltraine/configs/sample/icra_model.yaml__":

        ```bash
        $ python3 coiltraine.py --single-process train -e coil_icra --folder sample --gpus 0
        ```
        
        To Train all the models: 
        ```bash
        $ python3 coiltraine.py --folder sample --gpus 0
        ```
        
    2. __VALIDATE__
    
    Validation module produces one output for each of the data inputs from a some validation dataset. This store inside csv files located at "__~/.../coiltraine/_logs/<exp_batches>/<exp_alias>/<validation_dataset_name.csv>__"
    
    To run the validation for the as a __single process__, configs/sample/icra_model.yaml, with a certain validation dataset name:
    
    ```bash
    $ python3 coiltraine.py --single-process validation -e coil_icra --folder sample --gpus 0 -vd <validation_dataset>
    ```
    Note that the "validation_dataset" must be inside the folder defined in the __COIL_DATASET_PATH__ env variable.
    
    3. __DRIVE__
    
    Driving process is executed with the "__run_drive.py__" script. A different process is executed for every driving environnent (de) that is passed as parameter to the execution.
    
    For regular execution:
    
    ```bash
    $ python3 coiltraine.py --folder sample --gpus 0 -de <DrivingScenario_Town0X> --docker carlasim/carla:version
    ```
    Where __carlasim/carla:version__ is the installed docker version of CARLA.
    
    The "__DrivingScenario__" is a suite class defined inside the drive/suites folder. Each scenario defines the start and end positions for driving, the number of cars and pedestrians, weathers etc.
    
    The "__Town0X__" is the town used on the scenario and must mach its definition, for now it is either Town01 or Town02.

    __Program Execution for Driving__:

        1. __coiltraine.py__
            ```python
            elif args.single_process == 'drive':
                drive_params['suppress_output'] = False
                execute_drive("0", args.folder, args.exp, list(args.driving_environments)[0], drive_params)
            ```
        2. __executer.py__
            ```python
            def execute_drive(gpu, exp_batch, exp_alias, exp_set_name, params):

                params.update({'host': "127.0.0.1"})

                create_exp_path(exp_batch, exp_alias)
                p = multiprocessing.Process(target=run_drive.execute,
                                            args=(gpu, exp_batch, exp_alias, exp_set_name,
                                                  params))

                p.start()
            ```
        3. __run_drive.py__
            ```python
            def execute(gpu, exp_batch, exp_alias, drive_conditions, params):
                ...        
                    else:
                    """
                    #####
                    Main Loop , Run a benchmark for each specified checkpoint on the "Test Configuration"
                    #####
                    """
                    while not maximun_checkpoint_reach(latest, g_conf.TEST_SCHEDULE):
                        # Get the correct checkpoint
                        # We check it for some task name, all of then are ready at the same time
                        if is_next_checkpoint_ready(g_conf.TEST_SCHEDULE,
                                                    control_filename + '_' + task_list[0]):

                            latest = get_next_checkpoint(g_conf.TEST_SCHEDULE,
                                                         control_filename + '_' + task_list[0])

                            driving_benchmark(latest, gpu, town_name, experiment_set, exp_batch,
                                              exp_alias, params, control_filename, task_list)

                        else:
                            time.sleep(0.1)

            ```


### Exploring the Limitations of Behavior Cloning for Autonomous Driving [[Link](https://github.com/felipecode/coiltraine/blob/master/docs/exploring_limitations.md)]

Experiments from the paper

1. Models from the paper are available in "__~/.../coiltraine/configs/nocrash__" folder

2. Download the checkpoints
```bash
$ cd ~/.../coiltraine
$ python3 tools/download_nocrash_models.py
```

3. Reproducing Baseline results
To reproduce all the available models, using the "__gpu 0__", run:
```bash
$ cd ~/.../coiltraine
$ python3 coiltraine.py --gpus 0 --folder nocrash -de NocrashNewWeatherTown_Town02 NocrashNewWeather_Town01 NocrashTraining_Town01 NocrashNewTown_Town02 --docker carla_sim_ubuntu16:0.8.4
```

4. Test the best model on the hardest condition:
```bash
$ cd ~/.../coiltraine
$ python3 coiltraine.py --gpus 0 --single-process drive -e resnet34imnet10S2 --folder nocrash \
-de NocrashNewWeatherTown_Town02 --docker carla_sim_ubuntu16:0.8.4
```




<pre> <font color="#C3D82C">python3</font> <u style="text-decoration-style:single">coiltraine.py</u> --gpus 0 --single-process <u style="text-decoration-style:single">drive</u> -e resnet34imnet10S2 --folder nocrash \
-de NocrashNewWeatherTown_Town02 --docker carla_sim_ubuntu16:0.8.4
pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html
Running  /home/farnez/Projects/coiltraine/coil_core/run_drive.py  On GPU  0 of experiment name  resnet34imnet10S2
/home/farnez/Projects/coiltraine/configs/coil_global.py:102: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  yaml_file = yaml.load(f)
2020-07-16 14:01:12.154890: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2020-07-16 14:01:12.196665: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2020-07-16 14:01:12.197238: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1561] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: GeForce RTX 2080 computeCapability: 7.5
coreClock: 1.59GHz coreCount: 46 deviceMemorySize: 7.79GiB deviceMemoryBandwidth: 417.29GiB/s
2020-07-16 14:01:12.198393: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
2020-07-16 14:01:12.198444: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so.10
2020-07-16 14:01:12.216723: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcufft.so.10
2020-07-16 14:01:12.216923: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcurand.so.10
2020-07-16 14:01:12.217048: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcusolver.so.10
2020-07-16 14:01:12.223100: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcusparse.so.10
2020-07-16 14:01:12.223289: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudnn.so.7
2020-07-16 14:01:12.223596: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2020-07-16 14:01:12.225314: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2020-07-16 14:01:12.226698: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1703] Adding visible gpu devices: 0
2020-07-16 14:01:12.227987: I tensorflow/core/platform/cpu_feature_guard.cc:143] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
2020-07-16 14:01:12.237277: I tensorflow/core/platform/profile_utils/cpu_utils.cc:102] CPU Frequency: 2599990000 Hz
2020-07-16 14:01:12.237646: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7f7890000b20 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2020-07-16 14:01:12.237659: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
2020-07-16 14:01:12.321035: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2020-07-16 14:01:12.321893: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7c3cf00 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2020-07-16 14:01:12.321951: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): GeForce RTX 2080, Compute Capability 7.5
2020-07-16 14:01:12.322386: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2020-07-16 14:01:12.322656: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1561] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: GeForce RTX 2080 computeCapability: 7.5
coreClock: 1.59GHz coreCount: 46 deviceMemorySize: 7.79GiB deviceMemoryBandwidth: 417.29GiB/s
2020-07-16 14:01:12.322706: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
2020-07-16 14:01:12.322720: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so.10
2020-07-16 14:01:12.322733: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcufft.so.10
2020-07-16 14:01:12.322745: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcurand.so.10
2020-07-16 14:01:12.322755: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcusolver.so.10
2020-07-16 14:01:12.322769: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcusparse.so.10
2020-07-16 14:01:12.322780: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudnn.so.7
2020-07-16 14:01:12.322832: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2020-07-16 14:01:12.323106: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2020-07-16 14:01:12.323386: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1703] Adding visible gpu devices: 0
2020-07-16 14:01:12.323657: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
2020-07-16 14:01:12.325091: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1102] Device interconnect StreamExecutor with strength 1 edge matrix:
2020-07-16 14:01:12.325103: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1108]      0 
2020-07-16 14:01:12.325108: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1121] 0:   N 
2020-07-16 14:01:12.325506: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2020-07-16 14:01:12.325998: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2020-07-16 14:01:12.326270: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1247] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 6874 MB memory) -&gt; physical GPU (device: 0, name: GeForce RTX 2080, pci bus id: 0000:01:00.0, compute capability: 7.5)
 TESTED PATHJ  _logs/nocrash/resnet34imnet10S2/drive_NocrashNewWeatherTown_Town02_csv
/home/farnez/Projects/coiltraine/logger/monitorer.py:120: UserWarning: loadtxt: Empty input file: &quot;&lt;_io.BufferedReader name=&apos;_logs/nocrash/resnet34imnet10S2/drive_NocrashNewWeatherTown_Town02_csv/control_output_empty.csv&apos;&gt;&quot;
  delimiter=&quot;,&quot;, skiprows=1)
_logs/nocrash/resnet34imnet10S2/drive_NocrashNewWeatherTown_Town02_csv/control_output_empty.csv
_logs/nocrash/resnet34imnet10S2/drive_NocrashNewWeatherTown_Town02_csv/control_output_normal.csv
_logs/nocrash/resnet34imnet10S2/drive_NocrashNewWeatherTown_Town02_csv/control_output_cluttered.csv
Going to communicate
Checkpoint  480000
ERROR:root:(127.0.0.1:48849) connection closed
18 1 0
&lt;carla08.driving_benchmark.experiment.Experiment object at 0x7f7a40de2358&gt;
start index  5 end index 57
INFO:resnet34imnet10S2_drive_NocrashNewWeatherTown_Town02:{&apos;Iterating&apos;: {&apos;Checkpoint&apos;: 480000, &apos;Agent&apos;: &quot;tensor(0.0513, device=&apos;cuda:0&apos;, grad_fn=&lt;SelectBackward&gt;)&quot;}}
start index  77 end index 68
start index  70 end index 73
start index  46 end index 67
start index  34 end index 77
start index  61 end index 49
start index  21 end index 12
&lt;carla08.driving_benchmark.experiment.Experiment object at 0x7f7a40de22e8&gt;
start index  19 end index 66
start index  79 end index 14
start index  19 end index 57
start index  39 end index 53
start index  60 end index 26
start index  53 end index 76
start index  42 end index 13
start index  31 end index 71
start index  59 end index 35
start index  47 end index 16
start index  10 end index 61
start index  66 end index 3
start index  20 end index 79
start index  14 end index 56
start index  26 end index 69
start index  79 end index 19
start index  2 end index 29
start index  16 end index 14
start index  5 end index 57
start index  77 end index 68
start index  70 end index 73
start index  46 end index 67
start index  34 end index 77
start index  61 end index 49
start index  21 end index 12
&lt;carla08.driving_benchmark.experiment.Experiment object at 0x7f78fa8931d0&gt;
start index  19 end index 66
start index  79 end index 14
start index  19 end index 57
start index  39 end index 53
start index  60 end index 26
start index  53 end index 76
start index  42 end index 13
start index  31 end index 71
start index  59 end index 35
start index  47 end index 16
start index  10 end index 61
start index  66 end index 3
start index  20 end index 79
start index  14 end index 56
start index  26 end index 69
start index  79 end index 19
start index  2 end index 29
start index  16 end index 14
start index  5 end index 57
start index  77 end index 68
start index  70 end index 73
start index  46 end index 67
start index  34 end index 77
start index  61 end index 49
start index  21 end index 12
&lt;carla08.driving_benchmark.experiment.Experiment object at 0x7f78fa893128&gt;
start index  19 end index 66
start index  79 end index 14
start index  19 end index 57
start index  39 end index 53
start index  60 end index 26
start index  53 end index 76
start index  42 end index 13
start index  31 end index 71
start index  59 end index 35
start index  47 end index 16
start index  10 end index 61
start index  66 end index 3
start index  20 end index 79
start index  14 end index 56
start index  26 end index 69
start index  79 end index 19
start index  2 end index 29
start index  16 end index 14
start index  5 end index 57
start index  77 end index 68
start index  70 end index 73
start index  46 end index 67
start index  34 end index 77
start index  61 end index 49
start index  21 end index 12
&lt;carla08.driving_benchmark.experiment.Experiment object at 0x7f78fa893080&gt;
start index  19 end index 66
start index  79 end index 14
start index  19 end index 57
start index  39 end index 53
start index  60 end index 26
start index  53 end index 76
start index  42 end index 13
start index  31 end index 71
start index  59 end index 35
start index  47 end index 16
start index  10 end index 61
start index  66 end index 3
start index  20 end index 79
start index  14 end index 56
start index  26 end index 69
start index  79 end index 19
start index  2 end index 29
start index  16 end index 14
start index  5 end index 57
start index  77 end index 68
start index  70 end index 73
start index  46 end index 67
start index  34 end index 77
start index  61 end index 49
start index  21 end index 12
/home/farnez/Projects/coiltraine/carla08/driving_benchmark/metrics.py:340: RuntimeWarning: invalid value encountered in true_divide
  experiment_results_matrix[:, header.index(&apos;number_red_lights&apos;)]))).tolist()


----- Printing results for training weathers (Seen in Training) -----


Percentage of Successful Episodes

 VALUES 
  Average Between Weathers
    Task  0  -&gt;  0.0
    Task  1  -&gt;  0.0
    Task  2  -&gt;  0.0

Average Percentage of Distance to Goal Travelled 

 VALUES 
  Average Between Weathers
    Task  0  -&gt;  0.0
    Task  1  -&gt;  0.0
    Task  2  -&gt;  0.0

Average Percentage of Distance to Percentage OffRoad

 VALUES 
  Average Between Weathers
    Task  0  -&gt;  0.0
    Task  1  -&gt;  0.0
    Task  2  -&gt;  0.0

Average Percentage of Distance to Percentage Green Lights

 VALUES 
  Average Between Weathers
    Task  0  -&gt;  0.0
    Task  1  -&gt;  0.0
    Task  2  -&gt;  0.0

Avg. Kilometers driven before a collision to a PEDESTRIAN
  Average Between Weathers
    Task  0  -&gt; more than  0.0
    Task  1  -&gt; more than  0.0
    Task  2  -&gt; more than  0.0

Avg. Kilometers driven before a collision to a VEHICLE
  Average Between Weathers
    Task  0  -&gt; more than  0.0
    Task  1  -&gt; more than  0.0
    Task  2  -&gt; more than  0.0

Avg. Kilometers driven before a collision to a STATIC OBSTACLE
  Average Between Weathers
    Task  0  -&gt; more than  0.0
    Task  1  -&gt; more than  0.0
    Task  2  -&gt; more than  0.0

Avg. Kilometers driven before going OUTSIDE OF THE ROAD
  Average Between Weathers
    Task  0  -&gt; more than  0.0
    Task  1  -&gt; more than  0.0
    Task  2  -&gt; more than  0.0

Avg. Kilometers driven before invading the OPPOSITE LANE
  Average Between Weathers
    Task  0  -&gt; more than  0.0
    Task  1  -&gt; more than  0.0
    Task  2  -&gt; more than  0.0





----- Printing results for test weathers (Unseen in Training) -----


Percentage of Successful Episodes

 VALUES 
  Weather:  rainy after rain
    Task: 0  -&gt;  0.68
    Task: 1  -&gt;  0.48
    Task: 2  -&gt;  0.12
  Weather:  Soft Rain Sunset
    Task: 0  -&gt;  1.0
    Task: 1  -&gt;  0.88
    Task: 2  -&gt;  0.2
  Average Between Weathers
    Task  0  -&gt;  0.8400000000000001
    Task  1  -&gt;  0.6799999999999999
    Task  2  -&gt;  0.16

Average Percentage of Distance to Goal Travelled 

 VALUES 
  Weather:  rainy after rain
    Task: 0  -&gt;  0.5349211183489807
    Task: 1  -&gt;  0.5840984492995592
    Task: 2  -&gt;  -0.11035541501533219
  Weather:  Soft Rain Sunset
    Task: 0  -&gt;  0.9812186102077441
    Task: 1  -&gt;  0.8761437689312384
    Task: 2  -&gt;  0.24936191644813707
  Average Between Weathers
    Task  0  -&gt;  0.7580698642783624
    Task  1  -&gt;  0.7301211091153987
    Task  2  -&gt;  0.06950325071640244

Average Percentage of Distance to Percentage OffRoad

 VALUES 
  Weather:  rainy after rain
    Task: 0  -&gt;  0.008980020477385005
    Task: 1  -&gt;  0.011771328022199039
    Task: 2  -&gt;  0.0028016159060026396
  Weather:  Soft Rain Sunset
    Task: 0  -&gt;  0.0036214846845459103
    Task: 1  -&gt;  0.004584700096219024
    Task: 2  -&gt;  0.0020157339356027134
  Average Between Weathers
    Task  0  -&gt;  0.006300752580965458
    Task  1  -&gt;  0.00817801405920903
    Task  2  -&gt;  0.0024086749208026767

Average Percentage of Distance to Percentage Green Lights

 VALUES 
  Weather:  rainy after rain
    Task: 0  -&gt;  0.3
    Task: 1  -&gt;  0.446
    Task: 2  -&gt;  0.3906666666666667
  Weather:  Soft Rain Sunset
    Task: 0  -&gt;  0.30666666666666664
    Task: 1  -&gt;  0.4706666666666666
    Task: 2  -&gt;  0.4693333333333334
  Average Between Weathers
    Task  0  -&gt;  0.30333333333333334
    Task  1  -&gt;  0.4583333333333333
    Task  2  -&gt;  0.43000000000000005

Avg. Kilometers driven before a collision to a PEDESTRIAN
  Weather:  rainy after rain
    Task  0  -&gt; more than 7.404102774298396
    Task  1  -&gt;  6.7459667127596825
    Task  2  -&gt;  2.003911803922867
  Weather:  Soft Rain Sunset
    Task  0  -&gt; more than 8.835769915927957
    Task  1  -&gt; more than 8.451168569538183
    Task  2  -&gt;  2.5415565941411478
  Average Between Weathers
    Task  0  -&gt; more than  16.239872690226353
    Task  1  -&gt;  15.197135282297864
    Task  2  -&gt;  2.2727341990320076

Avg. Kilometers driven before a collision to a VEHICLE
  Weather:  rainy after rain
    Task  0  -&gt; more than 7.404102774298396
    Task  1  -&gt;  1.6864916781899206
    Task  2  -&gt;  0.30829412368044107
  Weather:  Soft Rain Sunset
    Task  0  -&gt; more than 8.835769915927957
    Task  1  -&gt; more than 8.451168569538183
    Task  2  -&gt;  0.31769457426764347
  Average Between Weathers
    Task  0  -&gt; more than  16.239872690226353
    Task  1  -&gt;  3.799283820574466
    Task  2  -&gt;  0.31348057917682864

Avg. Kilometers driven before a collision to a STATIC OBSTACLE
  Weather:  rainy after rain
    Task  0  -&gt; more than 7.404102774298396
    Task  1  -&gt;  3.3729833563798413
    Task  2  -&gt;  4.007823607845734
  Weather:  Soft Rain Sunset
    Task  0  -&gt; more than 8.835769915927957
    Task  1  -&gt;  8.451168569538183
    Task  2  -&gt;  5.0831131882822955
  Average Between Weathers
    Task  0  -&gt; more than  16.239872690226353
    Task  1  -&gt;  5.0657117607659545
    Task  2  -&gt;  4.545468398064015

Avg. Kilometers driven before going OUTSIDE OF THE ROAD
  Weather:  rainy after rain
    Task  0  -&gt; more than 7.404102774298396
    Task  1  -&gt; more than 6.7459667127596825
    Task  2  -&gt; more than 4.007823607845734
  Weather:  Soft Rain Sunset
    Task  0  -&gt; more than 8.835769915927957
    Task  1  -&gt; more than 8.451168569538183
    Task  2  -&gt; more than 5.0831131882822955
  Average Between Weathers
    Task  0  -&gt; more than  16.239872690226353
    Task  1  -&gt; more than  15.197135282297864
    Task  2  -&gt; more than  9.09093679612803

Avg. Kilometers driven before invading the OPPOSITE LANE
  Weather:  rainy after rain
    Task  0  -&gt;  0.6170085645248663
    Task  1  -&gt;  0.5621638927299736
    Task  2  -&gt;  0.6679706013076223
  Weather:  Soft Rain Sunset
    Task  0  -&gt;  1.2622528451325652
    Task  1  -&gt;  1.0563960711922729
    Task  2  -&gt;  1.2707782970705739
  Average Between Weathers
    Task  0  -&gt;  0.8547301415908607
    Task  1  -&gt;  0.7598567641148932
    Task  2  -&gt;  0.9090936796128031



 metric sum  [0. 2. 5.]
 metric sum  [ 0.  4. 30.]
 metric sum  [2. 3. 2.]
{&apos;episodes_fully_completed&apos;: array([[0.84],
       [0.68],
       [0.16],
       [0.  ],
       [0.  ],
       [0.  ]]), &apos;episodes_completion&apos;: array([[0.75806986],
       [0.73012111],
       [0.06950325],
       [0.        ],
       [0.        ],
       [0.        ]]), &apos;collision_pedestrians&apos;: array([[1.],
       [1.],
       [4.],
       [0.],
       [0.],
       [0.]]), &apos;collision_vehicles&apos;: array([[ 1.],
       [ 4.],
       [29.],
       [ 0.],
       [ 0.],
       [ 0.]]), &apos;collision_other&apos;: array([[1.],
       [3.],
       [2.],
       [0.],
       [0.],
       [0.]]), &apos;intersection_offroad&apos;: array([[1.],
       [1.],
       [1.],
       [0.],
       [0.],
       [0.]]), &apos;intersection_otherlane&apos;: array([[19.],
       [20.],
       [10.],
       [ 0.],
       [ 0.],
       [ 0.]]), &apos;end_pedestrian_collision&apos;: array([[0.  ],
       [0.04],
       [0.1 ],
       [0.  ],
       [0.  ],
       [0.  ]]), &apos;end_vehicle_collision&apos;: array([[0.  ],
       [0.08],
       [0.6 ],
       [0.  ],
       [0.  ],
       [0.  ]]), &apos;end_other_collision&apos;: array([[0.04],
       [0.06],
       [0.04],
       [0.  ],
       [0.  ],
       [0.  ]]), &apos;driven_kilometers&apos;: array([16.23987269, 15.19713528,  9.0909368 ]), &apos;average_speed&apos;: array([24.33851173])}
_logs/nocrash/resnet34imnet10S2/drive_NocrashNewWeatherTown_Town02_csv/control_output_empty.csv
_logs/nocrash/resnet34imnet10S2/drive_NocrashNewWeatherTown_Town02_csv/control_output_normal.csv
_logs/nocrash/resnet34imnet10S2/drive_NocrashNewWeatherTown_Town02_csv/control_output_cluttered.csv
[&apos;exp_id&apos;, &apos;rep&apos;, &apos;weather&apos;, &apos;start_point&apos;, &apos;end_point&apos;, &apos;collision_other&apos;, &apos;collision_pedestrians&apos;, &apos;collision_vehicles&apos;, &apos;intersection_otherlane&apos;, &apos;intersection_offroad&apos;, &apos;pos_x&apos;, &apos;pos_y&apos;, &apos;steer&apos;, &apos;throttle&apos;, &apos;brak&apos;]
end peds  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
success  [0. 1. 0. 0. 1. 0. 1. 1. 1. 0. 1. 1. 0. 1. 1. 1. 1. 1. 0. 1. 0. 1. 1. 1. 1. 0. 1. 0. 0. 1. 0. 0. 0. 1. 1. 1. 0. 0. 1. 0. 1. 1.
 1. 1. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 0. 1. 1. 1. 1. 1. 0.
 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0.]
End casues  150
[0. 1. 0. 4. 1. 4. 1. 1. 1. 0. 1. 1. 0. 1. 1. 1. 1. 1. 0. 1. 0. 1. 1. 1. 1. 3. 1. 0. 4. 1. 4. 0. 3. 1. 1. 1. 3. 0. 1. 2. 1. 1.
 1. 1. 1. 0. 3. 0. 0. 1. 3. 2. 0. 3. 3. 3. 3. 0. 3. 1. 2. 3. 3. 3. 3. 3. 1. 1. 0. 4. 3. 3. 3. 0. 2. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 4. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 2. 1. 1. 1. 1. 1. 3.
 3. 3. 3. 1. 3. 3. 2. 3. 1. 3. 3. 3. 1. 4. 3. 3. 3. 1. 1. 3. 3. 0. 3. 2.]
Number of episodes  149
Traceback (most recent call last):
  File &quot;/usr/lib/python3/dist-packages/PIL/Image.py&quot;, line 2428, in fromarray
    mode, rawmode = _fromarray_typemap[typekey]
KeyError: ((1, 1, 1), &apos;|u1&apos;)

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File &quot;/home/farnez/Projects/coiltraine/coil_core/run_drive.py&quot;, line 136, in driving_benchmark
    checkpoint_number, town_name, g_conf.PROCESS_NAME.split(&apos;_&apos;)[1])
  File &quot;/home/farnez/Projects/coiltraine/plotter/plot_on_map.py&quot;, line 242, in plot_episodes_tracks
    plot_test_image(map_image, os.path.join(paths_dir, str(checkpoint), str(count) + &apos;.png&apos;))
  File &quot;/home/farnez/Projects/coiltraine/plotter/plot_on_map.py&quot;, line 21, in plot_test_image
    image_to_plot = Image.fromarray(image.astype(&quot;uint8&quot;))
  File &quot;/usr/lib/python3/dist-packages/PIL/Image.py&quot;, line 2431, in fromarray
    raise TypeError(&quot;Cannot handle this data type&quot;)
TypeError: Cannot handle this data type
1d17ee463b9138c5c85c6489fc4c448275163ec6f754f99a05fcd7b912e44f11
INFO:resnet34imnet10S2_drive_NocrashNewWeatherTown_Town02:{&apos;Error&apos;: {&apos;Message&apos;: &apos;Something Happened&apos;}}
Traceback (most recent call last):
  File &quot;/usr/lib/python3/dist-packages/PIL/Image.py&quot;, line 2428, in fromarray
    mode, rawmode = _fromarray_typemap[typekey]
KeyError: ((1, 1, 1), &apos;|u1&apos;)

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File &quot;/home/farnez/Projects/coiltraine/coil_core/run_drive.py&quot;, line 136, in driving_benchmark
    checkpoint_number, town_name, g_conf.PROCESS_NAME.split(&apos;_&apos;)[1])
  File &quot;/home/farnez/Projects/coiltraine/plotter/plot_on_map.py&quot;, line 242, in plot_episodes_tracks
    plot_test_image(map_image, os.path.join(paths_dir, str(checkpoint), str(count) + &apos;.png&apos;))
  File &quot;/home/farnez/Projects/coiltraine/plotter/plot_on_map.py&quot;, line 21, in plot_test_image
    image_to_plot = Image.fromarray(image.astype(&quot;uint8&quot;))
  File &quot;/usr/lib/python3/dist-packages/PIL/Image.py&quot;, line 2431, in fromarray
    raise TypeError(&quot;Cannot handle this data type&quot;)
TypeError: Cannot handle this data type

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File &quot;/home/farnez/Projects/coiltraine/coil_core/run_drive.py&quot;, line 247, in execute
    exp_alias, params, control_filename, task_list)
  File &quot;/home/farnez/Projects/coiltraine/coil_core/run_drive.py&quot;, line 161, in driving_benchmark
    exit(1)
  File &quot;/usr/lib/python3.6/_sitebuiltins.py&quot;, line 26, in __call__
    raise SystemExit(code)
SystemExit: 1
INFO:resnet34imnet10S2_drive_NocrashNewWeatherTown_Town02:{&apos;Error&apos;: {&apos;Message&apos;: &apos;Something happened&apos;}}
</pre>

To fix the previous problem use the following package versions:

- pillow==5.3.0
- scikit-image==0.14.2
- imgaug==0.3.0


# My Experiments

## Docker images setup

Ubuntu NVIDIA CUDA Base Containers:

- [Container images](https://gitlab.com/nvidia/container-images/cuda/-/tree/master/dist)

- [ubuntu 16.04 - cuda 11.0](https://gitlab.com/nvidia/container-images/cuda/blob/master/dist/11.0/ubuntu16.04-x86_64/base/Dockerfile)

- [ubuntu 16.04 - cuda 10.2](https://gitlab.com/nvidia/container-images/cuda/-/blob/master/dist/10.2/ubuntu16.04-x86_64/base/Dockerfile)